In [10]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# figure out the correct path
machop_path = Path(".").resolve().parent.parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

from chop.tools.checkpoint_load import load_model
from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph.analysis import (
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
)
from chop.passes.graph import (
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.ir.graph.mase_graph import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")


INFO     Set logging level to info


In [11]:
batch_size = 512
model_name = "jsc-tiny"
dataset_name = "jsc"

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
    # custom_dataset_cache_path="../../chop/dataset"
)
data_module.prepare_data()
data_module.setup()

model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False,
    checkpoint = None)

# LAB1_CUSTOM_PATH = "/home/bkt123/dev/advanced-deep-learning-systems/mase/mase_output/lab-1_jsc-custom/software/training_ckpts/best.ckpt"
# model = load_model(load_name=LAB1_CUSTOM_PATH, load_type="pl", model=model)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
    max_batches=1
)

dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

In [12]:
# from chop.actions import train
# import torch

# # print(isinstance(mg.model, torch.nn.Module))

# model = mg.model
# model_info = get_model_info('jsc-tiny')
# dataset_info = get_dataset_info('jsc')
# task = "cls"

# train_params = {
#     "model": model,
#     "model_info": model_info,
#     "data_module": data_module,
#     "dataset_info": dataset_info,
#     "task": task,
#     "optimizer": "adam",
#     "learning_rate": 1e-3,
#     "weight_decay": 0,
#     "plt_trainer_args": {
#         "max_epochs": 1,
#     }, 
#     "auto_requeue": False,
#     "save_path": None,
#     "visualizer": None,
#     "load_name": None,
#     "load_type": None
# }

# train(**train_params)

In [13]:
from pprint import pprint

from chop.passes.graph.utils import get_mase_op

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

# pprint(mg.meta['mase'].__dict__)

for node in mg.fx_graph.nodes:
    #if get_mase_op(node) == 'conv2d':
    if node.op == "call_module":
        #print(node.name)
        print(50*'-')
        # print(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # print(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # print(node.meta['mase'].parameters['common']['args']['weight']['value'])
        # print(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])
        #pprint(mg.modules[node.target].weight)
        #pprint(mg.modules[node.target].state_dict())
        #pprint(mg.mo)
        print(50*'-')
print(mg.model.state_dict())

--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
OrderedDict([('seq_blocks.0.weight', tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])), ('seq_blocks.0.bias', tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])), ('seq_blocks.0.running_mean', tensor([-0.0141,  0.0178,  0.0146,  0.0144,  0.0177,  0.0154,  0.0067,  0.0064,
         0.0067,  0.0096,  0.0091,  0.0063,  0.0102,  0.0075,  0.0131,  0.0081])), ('seq_blocks.0.running_var', tensor([0.9675, 0.9589, 1.0103, 1.0214, 1.0143, 1.0225, 1.0039, 0.9967, 1.0039,
        0.9911, 0.9887, 0.9919, 0.9925, 0.9945, 1.0080, 0.9856])), ('seq_blocks.0

In [14]:
from chop.passes.graph.transforms import (
    prune_transform_pass,
)
pass_args = {
    "weight":{
        "scope" : "global",
        "granularity" : "elementwise",
        "method" :  "l1-norm",
        "sparsity" : 0.5,
    },
    "activation":{
        "scope" : "local",
        "granularity" : "elementwise",
        "method" : "l1-norm",
        "sparsity" : 0.5,
    },
}
 
mg, _ = prune_transform_pass(mg, pass_args)



In [15]:
from pprint import pprint

from chop.passes.graph.utils import get_mase_op

# pprint(mg.meta['mase'].__dict__)

for node in mg.fx_graph.nodes:
    #if get_mase_op(node) == 'linear':
    if node.op == "call_module":
        print(f"Layer: {node.name}")
        # pprint(node.meta['mase'].parameters['common'])
        # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # pprint(node.meta['mase'].parameters['common']['args']['weight']['value'])
        pprint(mg.modules[node.target])
        #pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
       # print(mg.model.state_dict())
        # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])
        #print(mg.modules[node.target].parametrizations['weight'][0].mask == mg.modules[node.target].parametrizations['weight'][1].mask)
        # total_w = 0
        # pruned_w = 0
        # w = mg.modules[node.target].weight
        # for s in w:
        #     total_w += s.numel()
        #     pruned_w += s.numel() - s.nonzero().numel()

        # pruned_percent = pruned_w / total_w
        # print(f"Pruned percent: {pruned_percent}")

        # print(50*'-')


Layer: seq_blocks_0
BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Layer: seq_blocks_1
ReLU(inplace=True)
Layer: seq_blocks_2
ParametrizedLinear(
  in_features=16, out_features=5, bias=True
  (parametrizations): ModuleDict(
    (weight): ParametrizationList(
      (0): FakeSparseWeight()
    )
  )
)
Layer: seq_blocks_3
ReLU(inplace=True)


In [16]:
from pprint import pprint

from chop.passes.graph.utils import get_mase_op

# pprint(mg.meta['mase'].__dict__)

for node in mg.fx_graph.nodes:
    if get_mase_op(node) == 'linear':
        print(f"Layer: {node.name}")
        # pprint(node.meta['mase'].parameters['common'])
        # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # pprint(node.meta['mase'].parameters['common']['args']['weight']['value'])
        #pprint(mg.modules[node.target].weight)
        # pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
        # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])

        # total_w = 0
        # pruned_w = 0
        # mask_2= mg.modules[node.target].parametrizations['weight'][0].mask
        # for s in mask_2:
        #     total_w += s.numel()
        #     pruned_w += s.numel() - s.nonzero().numel()

        # pruned_percent = pruned_w / total_w
        # print(f"Pruned percent: {pruned_percent}")

        print(50*'-')
print(mg.model.state_dict())

Layer: seq_blocks_2
--------------------------------------------------
OrderedDict([('seq_blocks.0.weight', tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])), ('seq_blocks.0.bias', tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])), ('seq_blocks.0.running_mean', tensor([-0.0141,  0.0178,  0.0146,  0.0144,  0.0177,  0.0154,  0.0067,  0.0064,
         0.0067,  0.0096,  0.0091,  0.0063,  0.0102,  0.0075,  0.0131,  0.0081])), ('seq_blocks.0.running_var', tensor([0.9675, 0.9589, 1.0103, 1.0214, 1.0143, 1.0225, 1.0039, 0.9967, 1.0039,
        0.9911, 0.9887, 0.9919, 0.9925, 0.9945, 1.0080, 0.9856])), ('seq_blocks.0.num_batches_tracked', tensor(2)), ('seq_blocks.2.bias', tensor([ 0.0921, -0.1772,  0.0255, -0.1234,  0.1454])), ('seq_blocks.2.parametrizations.weight.original', tensor([[ 0.0647,  0.0855, -0.1451, -0.2328,  0.1801,  0.0058,  0.2307, -0.0658,
          0.2478,  0.1071,  0.1862,  0.0013,  0.0230,  0.0391, -0.0746,  0.0537],
        [ 0

In [17]:
# from chop.actions import train
# import torch

# # print(isinstance(mg.model, torch.nn.Module))

# model = mg.model
# model_info = get_model_info('jsc-tiny')
# dataset_info = get_dataset_info('jsc')
# task = "cls"

# train_params = {
#     "model": model,
#     "model_info": model_info,
#     "data_module": data_module,
#     "dataset_info": dataset_info,
#     "task": task,
#     "optimizer": "adam",
#     "learning_rate": 1e-3,
#     "weight_decay": 0,
#     "plt_trainer_args": {
#         "max_epochs": 1,
#     }, 
#     "auto_requeue": False,
#     "save_path": None,
#     "visualizer": None,
#     "load_name": None,
#     "load_type": None
# }

# train(**train_params)

In [18]:
# from pprint import pprint

# from chop.passes.graph.utils import get_mase_op


# # pprint(mg.meta['mase'].__dict__)

# for node in mg.fx_graph.nodes:
#     if get_mase_op(node) == 'linear':
#         print(node.name)
#         print(50*'-')
#         # pprint(node.meta['mase'].parameters['common'])
#         # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
#         # pprint(node.meta['mase'].parameters['common']['args']['weight']['value'])
#         pprint(mg.modules[node.target].weight)
#         pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
#         # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])

#         print(50*'-')